# Tracer Advection Stencil

## Start a parallel cluster

In [1]:
import ipyparallel as ipp

cluster = ipp.Cluster(engines='mpi', n=6).start_and_connect_sync()

Starting 6 engines with <class 'ipyparallel.cluster.launcher.MPIEngineSetLauncher'>


  0%|          | 0/6 [00:00<?, ?engine/s]

In [2]:
%autopx

%autopx enabled


## Input parameters

In [48]:
%%capture
from mpi4py import MPI
import copy as cp
import functions as func
import numpy as np
import os
import importlib
importlib.reload(func)


mpi_comm = MPI.COMM_WORLD

backend = 'numpy'
layout = (1, 1)
nx, ny, nz, nhalo = 101, 101, 80, 3   # number of points in x, y, z dimensions, halos
nx, ny, nz, nhalo = 12, 12, 79, 3     # number of points in x, y, z dimensions, halos

dt_acoustic = 300.                    # acoustic time step in seconds
advection_timestep = 3600.            # advection time step
nSteps_advection = 500                # number of advection steps to be run
print_advectionProgress = True        # True: print timestep differences on rank 0
plot_output = True                    # True: plot output 
figure_everyNsteps = 100              # how often to plot a figure
write_initialCondition = True         # True: write out initial condition and coordinate system

# finite volume transport options
grid_type = 0                         # cubed-sphere
hord = 6                              # horizontal advection order

# initial condition adjustables
pressure_base = 10                    # pressure thickness of layer in Pa
smoke_base = 1                        # maximum smoke amount
smoke_center_tile = 0                 # rank of tile on which to center the gaussian tracer
density = 1                           # density of air for mass flux


######-------- this can be left alone --------######
# frequency of storing and plotting tracer state
if figure_everyNsteps > nSteps_advection:
    figure_everyNsteps = nSteps_advection

tracer_archive_frequency = figure_everyNsteps

pwd = os.getcwd()
nx_effective, ny_effective, nz_effective = nx + 1, ny + 1, nz + 1

## Prepare state for advection

In [50]:
%%capture
dimensions, units, origins, rank = func.define_dimensionsUnitsOrigins(nx_effective, ny_effective, nz_effective, nhalo, mpi_comm)
smoke_dict = {'center_tile': smoke_center_tile, 'rank': rank, 'smoke_base': smoke_base}
pressure_dict = {'pressure_base': pressure_base}
fvt_dict = {'grid_type': grid_type, 'hord': hord}

configuration = func.configure_domain(layout, mpi_comm, dimensions, backend=backend)
initialState = func.create_initialState_testCase1(configuration['grid_data'], dimensions, units, origins, 
                                                  backend, smoke_dict, pressure_dict)

tracAdv_data, tracAdv = func.prepare_everythingForAdvection(configuration, initialState['uC'], initialState['vC'], 
                                                            initialState['delp'], {'smoke': initialState['smoke']}, 
                                                            density, dimensions, units, origins, backend, 
                                                            fvt_dict, dt_acoustic=dt_acoustic)
                                                            
tracAdv_dataInit = cp.deepcopy(tracAdv_data)


if write_initialCondition:
    saveDir = pwd + '/data/'
    fOut = saveDir + 'initialState_%sx%s.nc' % (dimensions['nx']-1, dimensions['ny']-1)
            
    func.write_initialCondition_toFile(fOut, initialState, dimensions, units, origins, backend, configuration, rank)


%px:   0%|          | 0/6 [00:00<?, ?tasks/s]

## Advection loop

In [5]:
#%%capture
tracer_archive = [cp.deepcopy(tracAdv_data['tracers']['smoke'])]
timestep_archive = [0]
for step in range(nSteps_advection):    
    if (rank == 0) and (step+1)%100 == 0 and print_advectionProgress:
        print('Step: %d' % (step+1))
    #tracAdv_data = func.run_advectionStepWithReset(tracAdv_dataInit, tracAdv_data, tracAdv, advection_timestep, mpi_rank=rank)
    tracAdv_data = func.run_advectionStepWithReset(tracAdv_dataInit, tracAdv_data, tracAdv, advection_timestep)

    if (step+1)%tracer_archive_frequency == 0:
        tracer_archive.append(cp.deepcopy(tracAdv_data['tracers']['smoke']))
        timestep_archive.append(step+1)

%px:   0%|          | 0/6 [00:00<?, ?tasks/s]

[stdout:0] Step: 100
Step: 200
Step: 300
Step: 400
Step: 500


## Plot output

In [6]:
#%%capture
if plot_output:
    lon, lat = func.get_lonLat_edges(configuration['grid_data'], dimensions, units, origins, backend)
    lon_global = configuration['communicator'].gather(lon)
    lat_global = configuration['communicator'].gather(lat)
    smoke_global = []
    for step in range(len(timestep_archive)):
        smoke_global.append(configuration['communicator'].gather(tracer_archive[step]))

    if rank == 0:
        saveDir = pwd + '/figs/'
        fOut = saveDir + 'advection_step_'
        
        if not os.path.isdir(saveDir):
            os.mkdir(saveDir)
        else:
            fls = os.listdir(saveDir)
            if len(fls) > 0:
                for fl in fls:
                    os.remove(saveDir + fl)

        smoke_global = np.squeeze(np.stack(smoke_global))
        for plotStep in range(len(timestep_archive)):
            print('Plotting time step %s / %s' % (timestep_archive[plotStep], nSteps_advection))
            func.plot_projection_field(lon_global.data, lat_global.data, smoke_global[plotStep], 
                                       cmap='viridis', vmin=0, vmax=1, units='kg', 
                                       title='Smoke after %s advection steps' % timestep_archive[plotStep], 
                                       fSave=fOut+'{:03d}'.format(timestep_archive[plotStep]) + '.png')

[stdout:0] Plotting time step 0 / 500
Plotting time step 100 / 500
Plotting time step 200 / 500
Plotting time step 300 / 500
Plotting time step 400 / 500
Plotting time step 500 / 500


In [7]:
%autopx

%autopx disabled


In [8]:
cluster.shutdown()